## EMMA step 2: Projecting streamwater samples and potential endmembers into PCA space to select endmembers for EMMA
### -  Tracers selected in EMMA step 1: bivariate plots to ID conservative mixing

###  - MED note that my ealier python code had a mistake in that separate PCAs were performed on the streamwater and EM datasets. We want the same coordinate system to determine their relationships, so this code rectifies that issue. As in EMMALAB, the EM data are projected into the streamwater PCA space.

### - Dynamic read-in: to use this notebook, change the 'site' parameter to "Wade" or "Hungerford". Tracers and sample filtering will then occur as needed.

### - Event-based PCAs: take as function parameters: site, date range, EM1, EM2, EM3

In [5]:
#####################################
# PCA of streamwater and EMs ########
# Megan E Duffy 07/18/2025 ##########
# Part of EMMA workflow #############
# EVENT SPECIFIC PCA FOR EMMA #######
#####################################

# Set working directory
import os
os.chdir("/home/millieginty/OneDrive/git-repos/EMMA/")

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def plot_event_pca(
    data,
    site,
    start_date,
    end_date,
    endmember_ids,
    title="Event-Specific PCA"
):
    """
    Generate PCA plot for a specific storm event, following the EMMALAB workflow:
    PCA is fit ONLY on streamwater (mixture) data, then endmembers are projected
    into that PCA space.

    Parameters:
        data (DataFrame): Full dataframe containing stream and endmember samples.
        site (str): Site name ("Wade" or "Hungerford").
        start_date (str or datetime): Start of storm event (e.g., "2023-04-01").
        end_date (str or datetime): End of storm event (e.g., "2023-04-04").
        endmember_ids (list of str): List of Sample IDs to use as endmembers.
        title (str): Title for the plot.
    """

    # Site-specific tracers
    if site == "Wade":
        tracers = ['Ca_mg_L', 'Si_mg_L', 'Mg_mg_L', 'dD', 'd18O', 'Na_mg_L']
    elif site == "Hungerford":
        tracers = ['Ca_mg_L', 'Cl_mg_L', 'Si_mg_L', 'Na_mg_L', 'Mg_mg_L', 'dD', 'd18O']
    else:
        raise ValueError("Site not recognized. Use 'Wade' or 'Hungerford'.")

    # Ensure datetime column is datetime type
    data['Datetime'] = (data['Date'] + ' ' + data['Time']) # Combine the strings of original inventory Date and Time cols
    data['Datetime'] = pd.to_datetime(data['Datetime'], format="%m/%d/%Y %H:%M", errors="coerce") 
    data = data[data['Datetime'].notna()] # NA dates (we have a couple in the RI23 dataset) not useful - prune 

    # Subset streamwater (mixture) in date range and site
    stream = data[
        (data["Site"] == site) &
        (data["Type"].isin(["Grab", "Isco", "Baseflow"])) &
        (data["Datetime"] >= pd.to_datetime(start_date)) &
        (data["Datetime"] <= pd.to_datetime(end_date))
    ].copy()

    # Subset endmembers by Sample ID
    endmembers = data[
        (data["Site"] == site) &
        (data["Sample ID"].isin(endmember_ids))
    ].copy()

    # Drop NA for stream tracers
    subset_stream = stream[tracers].dropna().copy()
    subset_stream["Group"] = "Streamwater"
    subset_stream["Type"] = "Streamwater"
    subset_stream["Datetime"] = stream["Datetime"]

    # Fill NA for endmembers with mean (per tracer)
    subset_endmembers = endmembers[tracers].copy()
    subset_endmembers = subset_endmembers.fillna(subset_endmembers.mean())
    subset_endmembers["Group"] = "Endmember"
    subset_endmembers["Type"] = endmembers["Type"].values
    subset_endmembers["Datetime"] = endmembers["Datetime"].values

    # -----------------------
    # PCA logic 
    # -----------------------

    # 1. Fit scaler ONLY on streamwater using fit_transform
    scaler = StandardScaler()
    scaled_stream = scaler.fit_transform(subset_stream[tracers])

    # 2. PCA ONLY on scaled streamwater using pca.fit_transform
    pca = PCA(n_components=2)
    stream_pca_result = pca.fit_transform(scaled_stream)
    subset_stream["PC1"] = stream_pca_result[:, 0]
    subset_stream["PC2"] = stream_pca_result[:, 1]

    # 3. Project endmembers using same scaler & PCA using and transform and pca.transform
    scaled_endmembers = scaler.transform(subset_endmembers[tracers])
    endmember_pca_result = pca.transform(scaled_endmembers)
    subset_endmembers["PC1"] = endmember_pca_result[:, 0]
    subset_endmembers["PC2"] = endmember_pca_result[:, 1]

    # Combine for plotting
    combined = pd.concat([subset_stream, subset_endmembers], ignore_index=True)

    # -----------------------
    # Plotting
    # -----------------------
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot streamwater points
    sw = combined[combined["Group"] == "Streamwater"]
    ax.scatter(sw["PC1"], sw["PC2"], marker='+', c='blue', alpha=0.5, label='Streamwater')

    # Plot endmembers with distinct markers/colors
    endmember_markers = {
        'Rain': 'o', 'Snow': 's', 'Snowmelt lysimeter': '^', 'Precip': '*',
        'Soil water lysimeter dry': '<', 'Soil water lysimeter wet': '>',
        'Groundwater': 'D', 'Baseflow': 'P'
    }
    colors = ['#d7191c', '#fdae61', '#abdda4', '#2b83ba', '#2ca25f', '#636363', '#8856a7', '#d95f0e']
    for (etype, color) in zip(endmember_markers.keys(), colors):
        em = combined[(combined["Type"] == etype)]
        if not em.empty:
            ax.scatter(em["PC1"], em["PC2"],
                       marker=endmember_markers[etype], c=color, edgecolors='black',
                       alpha=0.8, s=100, label=etype)
            for _, row in em.iterrows():
                if pd.notnull(row["Datetime"]):
                    ax.text(row["PC1"], row["PC2"], row["Datetime"].strftime('%m/%d'),
                            fontsize=12, ha='right')

    # Variance explained for axis labels
    pc1_var = pca.explained_variance_ratio_[0] * 100
    pc2_var = pca.explained_variance_ratio_[1] * 100
    ax.set_xlabel(f"PC1 ({pc1_var:.1f}%)")
    ax.set_ylabel(f"PC2 ({pc2_var:.1f}%)")

    ax.set_title(title)

    # --- Save plot as 'output/site_title.jpg' ---
    # Sanitize filename: replace spaces and parentheses
    save_event_name = title.replace(" ", "_").replace("(", "").replace(")", "").replace(":", "")
    filename = f"{site}_{save_event_name}.jpg"
    output_path = os.path.join("output", filename)

    fig.savefig(output_path, dpi=300)
    print(f"Saved plot to: {output_path}")
    
    ax.legend(bbox_to_anchor=(1.02, 1.02), loc="upper left")
    plt.tight_layout()
    plt.show()

In [6]:
# Load the full RI23 dataset
df = pd.read_csv("data/newrnet-chemistry/RI24/RI24-IC-ICP-isotope-joined.csv")

####################
# Wade RI24 events #
####################

# Wade RI24 series
# Endmembers: 
    # RI23-5018 Groundwater Bear Pond 03/10/23
    # RI23-5006 Groundwater High Pond 03/01/23 (could be 03/10/23, doesn't really matter)
    # RI23-5000 Soil water lysimeter dry 02/15/23
    # RI23-5005 Snowmelt lysimeter 2/15
    # RI23-1032 Baseflow (pre-event 2/17, really post event..)

plot_event_pca(
    data=df,
    site="Wade",
    start_date="2024-01-01",
    end_date="2024-05-01",
    endmember_ids=["RI24-1418", "RI24-1419", "RI24-1420", "RI24-1421"],  
    title="PCA for Wade RI24 series"
)

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required by StandardScaler.

In [7]:
df

,Sample ID,Site,Date,Time,Time Zone,Type,Type2,Freezer bag #,UVM ICPOES prep date,Index-notes,...,Si_mg_L,Sr_mg_L,Zn_mg_L,Cu_mg_L,Cl_mg_L,IC-notes_MED,dD,d18O,NRS_LWIA_notes,Datetime
0,RI24-1418,Wade,12/19/2023,11:00:00 AM,EST,Snowmelt lysimeter,Endmember,NaN,05/21/2024,Dry site lysimeter,...,1.605420,0.028877,0.143111,NaN,2.958,Run 2025-07-21,-123.170,-17.298,NaN,NaT
1,RI24-1419,Wade,12/19/2023,11:30:00 AM,EST,Soil water lysimeter,Endmember,NaN,05/21/2024,Wet site lysimeter,...,1.271612,0.013356,0.116164,NaN,3.177,Run 2025-07-21,-60.017,-9.172,NaN,NaT
2,RI24-1420,Wade,12/19/2023,12:00:00 PM,EST,Soil water lysimeter,Endmember,NaN,05/21/2024,Dry site lysimeter,...,1.062430,0.006098,0.443366,NaN,3.041,Run 2025-07-21,-100.162,-14.696,NaN,NaT
3,RI24-1415,Hungerford,12/20/2023,09:30:00 AM,EST,Snowmelt lysimeter,Endmember,NaN,05/21/2024,Wet site lysimeter,...,1.417292,0.025778,0.150549,NaN,4.043,Run 2025-06-05,-123.809,-17.231,NaN,NaT
4,RI24-1416,Hungerford,12/20/2023,09:45:00 AM,EST,Soil water lysimeter,Endmember,NaN,05/21/2024,Wet site lysimeter,...,NaN,NaN,NaN,NaN,5.064,Run 2025-07-21,-62.853,-9.877,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,RI24-1414,Hungerford,04/14/2024,02:30:00 PM,EDT,Grab,Stream,NaN,NaN,Grab Taken in (acidwashed) polypro bottle,...,NaN,NaN,NaN,NaN,NaN,NaN,-86.118,-12.429,NaN,NaT
418,RI24-1409,Wade,04/14/2024,02:00:00 AM,EDT,Isco,Stream,NaN,07/12/2024,NaN,...,1.963619,0.012497,0.005577,NaN,NaN,NaN,NaN,NaN,NaN,NaT
419,RI24-1410,Wade,04/14/2024,05:00:00 AM,EDT,Isco,Stream,NaN,07/12/2024,NaN,...,1.907754,0.012911,0.003386,NaN,NaN,NaN,-104.411,-15.212,NaN,NaT
420,RI24-1411,Wade,04/14/2024,08:00:00 AM,EDT,Isco,Stream,NaN,07/12/2024,NaN,...,5.357178,0.185374,0.003039,NaN,NaN,NaN,-104.173,-15.105,NaN,NaT


In [4]:
## Same but with average PC scores for same type EMs

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def plot_event_pca_with_avgEM(
    data,
    site,
    start_date,
    end_date,
    endmember_ids,
    title="Event-Specific PCA"
):
    """
    Generate PCA plot for a specific storm event, following the EMMALAB workflow:
    PCA is fit ONLY on streamwater (mixture) data, then endmembers are projected
    into that PCA space.
    Also plots AvgEMScore: mean PC score for each endmember type.
    """

    # Site-specific tracers
    if site == "Wade":
        tracers = ['Ca_mg_L', 'Si_mg_L', 'Mg_mg_L', 'dD', 'd18O', 'Na_mg_L']
    elif site == "Hungerford":
        tracers = ['Ca_mg_L', 'Cl_mg_L', 'Si_mg_L', 'Na_mg_L', 'Mg_mg_L', 'dD', 'd18O']
    else:
        raise ValueError("Site not recognized. Use 'Wade' or 'Hungerford'.")

    # Ensure datetime column is datetime type
    data["Date"] = pd.to_datetime(data["Date"], format="%m/%d/%Y", errors="coerce")

    # Subset streamwater (mixture) in date range and site
    stream = data[
        (data["Site"] == site) &
        (data["Type"].isin(["Grab", "Grab/Isco", "Baseflow"])) &
        (data["Date"] >= pd.to_datetime(start_date)) &
        (data["Date"] <= pd.to_datetime(end_date))
    ].copy()

    # Subset endmembers by Sample ID
    endmembers = data[
        (data["Site"] == site) &
        (data["Sample ID"].isin(endmember_ids))
    ].copy()

    # Drop NA for stream tracers
    subset_stream = stream[tracers].dropna().copy()
    subset_stream["Group"] = "Streamwater"
    subset_stream["Type"] = "Streamwater"
    subset_stream["Date"] = stream["Date"]

    # Fill NA for endmembers with mean (per tracer)
    subset_endmembers = endmembers[tracers].copy()
    subset_endmembers = subset_endmembers.fillna(subset_endmembers.mean())
    subset_endmembers["Group"] = "Endmember"
    subset_endmembers["Type"] = endmembers["Type"].values
    subset_endmembers["Date"] = endmembers["Date"].values

    # -----------------------
    # EMMALAB PCA logic
    # -----------------------
    scaler = StandardScaler()
    scaled_stream = scaler.fit_transform(subset_stream[tracers])

    pca = PCA(n_components=2)
    stream_pca_result = pca.fit_transform(scaled_stream)
    subset_stream["PC1"] = stream_pca_result[:, 0]
    subset_stream["PC2"] = stream_pca_result[:, 1]

    scaled_endmembers = scaler.transform(subset_endmembers[tracers])
    endmember_pca_result = pca.transform(scaled_endmembers)
    subset_endmembers["PC1"] = endmember_pca_result[:, 0]
    subset_endmembers["PC2"] = endmember_pca_result[:, 1]

    # Combine
    combined = pd.concat([subset_stream, subset_endmembers], ignore_index=True)

    # -----------------------
    # Compute AvgEMScore
    # -----------------------
    avg_em = (
        subset_endmembers.groupby("Type")[["PC1", "PC2"]]
        .mean()
        .reset_index()
    )
    avg_em["Group"] = "AvgEMScore"

    # -----------------------
    # Plotting
    # -----------------------
    fig, ax = plt.subplots(figsize=(10, 6))

    # Streamwater points
    sw = combined[combined["Group"] == "Streamwater"]
    ax.scatter(sw["PC1"], sw["PC2"], marker='+', c='blue', alpha=0.5, label='Streamwater')

    # Endmember markers/colors
    endmember_markers = {
        'Rain': 'o', 'Snow': 's', 'Snowmelt lysimeter': '^', 'Precip': '*',
        'Soil water lysimeter dry': '<', 'Soil water lysimeter wet': '>',
        'Groundwater': 'D', 'Baseflow': 'P'
    }
    colors = ['#d7191c', '#fdae61', '#abdda4', '#2b83ba',
              '#2ca25f', '#636363', '#8856a7', '#d95f0e']

    # Plot individual endmembers
    for (etype, color) in zip(endmember_markers.keys(), colors):
        em = combined[(combined["Type"] == etype)]
        if not em.empty:
            ax.scatter(em["PC1"], em["PC2"],
                       marker=endmember_markers[etype], c=color, edgecolors='black',
                       alpha=0.5, s=80, label=f"{etype} sample")
            for _, row in em.iterrows():
                if pd.notnull(row["Date"]):
                    ax.text(row["PC1"], row["PC2"], row["Date"].strftime('%m/%d'),
                            fontsize=10, ha='right')

    # Plot AvgEMScore as larger stars
    for (etype, color) in zip(endmember_markers.keys(), colors):
        em_avg = avg_em[avg_em["Type"] == etype]
        if not em_avg.empty:
            ax.scatter(em_avg["PC1"], em_avg["PC2"],
                       marker='*', c=color, edgecolors='black',
                       alpha=1.0, s=200, label=f"{etype} AvgEM")

    # Variance explained
    pc1_var = pca.explained_variance_ratio_[0] * 100
    pc2_var = pca.explained_variance_ratio_[1] * 100
    ax.set_xlabel(f"PC1 ({pc1_var:.1f}%)")
    ax.set_ylabel(f"PC2 ({pc2_var:.1f}%)")

    ax.set_title(title)
    ax.legend(bbox_to_anchor=(1.02, 1.02), loc="upper left")
    plt.tight_layout()
    plt.show()
